# Объединённый ноутбук: Генерация и анализ арифметических выражений

В этом ноутбуке выполняются все задачи по генерации, визуализации, классификации и кодированию арифметических выражений на основе изображений из папки `data`.

In [ ]:
import os
import random
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.neighbors import KNeighborsClassifier
import re
import string


## 1. Генерация случайного выражения
Функция `create_expression_sequence` формирует список символов: возможно скобки, операнд (цифра или буква), оператор, второй операнд и, возможно, закрывающая скобка.

In [ ]:
def create_expression_sequence():
    seq = []
    # Открывающая скобка (иногда)
    if random.choice([True, False]):
        seq.append('(')
    # Первое число или буква
    seq.append(random.choice(list(string.digits + string.ascii_letters)))
    # Оператор
    seq.append(random.choice(['+', '-', '*', '/']))
    # Второе число или буква
    seq.append(random.choice(list(string.digits + string.ascii_letters)))
    # Закрывающая скобка (иногда)
    if random.choice([True, False]):
        seq.append(')')
    return seq

# Пример использования
expression = create_expression_sequence()
print("Символы выражения:", expression)


## 2. Визуализация символов
Функции для выбора случайного изображения символа из папок `data/<символ>` и для объединения их в одно изображение выражения.

In [ ]:
def fetch_random_image(symbol):
    folder = os.path.join('data', symbol)
    choice = random.choice(os.listdir(folder))
    img = cv2.imread(os.path.join(folder, choice))
    return cv2.resize(img, (45, 45))

def build_expression_image(seq, output_path='expression.png'):
    images = [fetch_random_image(sym) for sym in seq]
    combined = np.hstack(images)
    cv2.imwrite(output_path, combined)
    return combined

# Визуализируем
expr_img = build_expression_image(expression, 'generated_expression.png')
print("Сохранено изображение выражения: generated_expression.png")


## 3. Обучение KNN-классификатора
Собираем признаки HOG для нескольких примеров каждого класса и обучаем `KNeighborsClassifier`. Возьмём по 15 изображений каждого символа.

In [ ]:
def train_knn_model(k=3, samples_per_class=15):
    X, y = [], []
    classes = os.listdir('data')
    for cls in classes:
        files = random.sample(os.listdir(os.path.join('data', cls)), samples_per_class)
        for f in files:
            img = cv2.imread(os.path.join('data', cls, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (45, 45))
            features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
            X.append(features)
            y.append(cls)
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X, y)
    return model

knn = train_knn_model()
print("KNN-модель обучена.")

## 4. Классификация выражения
Разбиваем изображение выражения на фрагменты, вычисляем HOG-признаки и восстанавливаем последовательность символов.

In [ ]:
def classify_expression_image(img, model):
    h, w = img.shape[:2]
    n = w // 45
    seq = []
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    for i in range(n):
        crop = gray[:, i*45:(i+1)*45]
        feat = hog(crop, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
        seq.append(model.predict([feat])[0])
    return seq

predicted_seq = classify_expression_image(cv2.imread('generated_expression.png'), knn)
expr_str = ''.join(predicted_seq)
print("Восстановленная строка:", expr_str)


## 5. Проверка корректности и вычисление
Используем регулярные выражения для валидации и `eval` для получения результата. Буквы преобразуем в их ASCII-коды.

In [ ]:
def validate_expression(expr):
    pattern = r'^\(?[A-Za-z0-9][\+\-\*\/][A-Za-z0-9]\)?$'
    return bool(re.match(pattern, expr))

def evaluate_expression(expr):
    # Заменяем буквы на их ASCII-коды
    expr_trans = ''.join(str(ord(c)) if c.isalpha() else c for c in expr)
    return eval(expr_trans)

if validate_expression(expr_str):
    result = evaluate_expression(expr_str)
    print("Результат выражения:", result)
else:
    print("Строка не является корректным выражением.")

## 6. Кодирование в бинарную строку
Преобразуем итоговое изображение в бинарный формат и сохраняем в файл `binary_output.txt`.

In [ ]:
def encode_to_binary(file_path='generated_expression.png', output_txt='binary_output.txt'):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    flat = (thresh.flatten() > 0).astype(int)
    bin_str = ''.join(map(str, flat))
    with open(output_txt, 'w') as f:
        f.write(bin_str)
    print(f"Бинарная строка сохранена в {output_txt}")

encode_to_binary()